# **Estimate Roof Greening Potenial**

In [ ]:
# Sometimes encoding needs to be specified because of a Google Colab bug
import locale

locale.getpreferredencoding = lambda: "UTF-8"
print(locale.getpreferredencoding())

## **Install dependencies**

In [ ]:
%pip install pypdl rasterio localtileserver rasterstats

## **Specify working directory**

Input your working directory path and choose whether you will work using Google Drive or locally

In [ ]:
working_dir = '/My Drive/Thesis_Finn_New'
work_on_drive = True

Mount your Google Drive

In [ ]:
from google.colab import drive
import os


if work_on_drive == True:
  drive_dir = '/content/drive'
  drive.mount(drive_dir)
  os.chdir(drive_dir + working_dir)
else:
  os.chdir(working_dir)

## **Import the required modules**

In [ ]:
import csv
import geopandas as gpd
import numpy as np
import rasterio
import zipfile

from modules.roof_processing import *
from osgeo import gdal
from pypdl import Downloader
from rasterio.merge import merge
from rasterstats import zonal_stats

## **Download required files**

Download the required BAG, DTM, and DSM from PDOK if they are missing. You can also import them yourself (use the same names as specified here).

In [ ]:
# Prepare folder structure
prepare_roof_folders()

# Download the BAG (building data) and municipal boundaries
BAG_path = os.path.join("roofs", "BAG", "bag-light.gpkg")
boundaries_zip_path = os.path.join("roofs", "Boundaries", "municipal_boundaries.zip")
boundaries_path = os.path.join("roofs", "Boundaries", "bestuurlijkegrenzen.gpkg")

dl = Downloader()

if not os.path.isfile(BAG_path):
  print("Downloading BAG...")
  dl.start('https://service.pdok.nl/lv/bag/atom/downloads/bag-light.gpkg', BAG_path)
if not os.path.isfile(boundaries_zip_path):
  print("Downloading municipal boundaries...")
  dl.start('https://service.pdok.nl/kadaster/bestuurlijkegrenzen/atom/v1_0/downloads/bestuurlijkegrenzen_gpkg_2021.zip', boundaries_zip_path)
if not os.path.isfile(boundaries_path):
  # Extract the municipal boundaries geopackage
  with zipfile.ZipFile(boundaries_zip_path, 'r') as boundaries_zip:
    print("Extracting boundaries geopackage...")
    extract_path = os.path.join("roofs", "Boundaries")

    # Extracts file called 'bestuurlijkegrenzen.gpkg'
    boundaries_zip.extractall(extract_path)

print("BAG and municipal boundaries have been downloaded")

Download Amsterdam DSM and DTM tiles

In [ ]:
# These text files contain what tiles to download from PDOK
DSM_info_path = os.path.join("roofs", "Tile_info", "DSM_Tile_Info.txt")
DTM_info_path = os.path.join("roofs", "Tile_info", "DTM_Tile_Info.txt")

# Download DSM and DTM tiles
download_DSM_DTM_tiles(dl, DSM_info_path, DTM_info_path)

## **Preprocess data**

Load BAG (building data)

In [ ]:
# Read the Amsterdam boundaries and BAG files
gdf_boundaries = gpd.read_file(boundaries_path)

BAG_feather_path = os.path.join("roofs", "BAG", "BAG.feather")

# Create a feather file which can load the BAG quicker in the future
if os.path.isfile(BAG_feather_path):
  print("Loading BAG as Feather file...")
  gdf_BAG = gpd.read_feather(BAG_feather_path)
else:
  print("Creating BAG Feather file...")
  gdf_BAG = gpd.read_file(BAG_path, layer='pand')
  gdf_BAG.to_feather(BAG_feather_path)

# Limit BAG to Amsterdam
BAG_Amsterdam_gdf = gpd.sjoin(gdf_BAG, gdf_boundaries[gdf_boundaries['gemeentenaam'] == 'Amsterdam'], how="inner", predicate="within")

# Limit BAG to only buildings that are already built and (planned to be) in use
allowed_statuses = ['Pand in gebruik', 'Pand in gebruik (niet ingemeten)', 'Verbouwing pand']
BAG_Amsterdam_gdf = BAG_Amsterdam_gdf[BAG_Amsterdam_gdf['status'].isin(allowed_statuses)]

print("BAG is ready")

Merge the DSM tiles to a new file (if it doesn't exist yet). This is used for the roof slope calculation.

Also merge the DTM tiles to a new file. This is used for the building height calculation.

In [ ]:
# Merge DSM tiles to DSM mosaic
DSM_tiles_path = os.path.join("roofs", "DSM_tiles")
DSM_merged_path = os.path.join("roofs", "DSM_merged", "Amsterdam_DSM.tif")

if not os.path.isfile(DSM_merged_path):
  merge_tif_files(DSM_tiles_path, DSM_merged_path)
  print('Amsterdam_DSM.tif has been created')
else:
  print("Amsterdam_DSM.tif already exists")

# Merge DTM tiles to DTM mosaic
DTM_tiles_path = os.path.join("roofs", "DTM_tiles")
DTM_merged_path = os.path.join("roofs", "DTM_merged", "Amsterdam_DTM.tif")

if not os.path.isfile(DTM_merged_path):
  merge_tif_files(DTM_tiles_path, DTM_merged_path)
  print('Amsterdam_DTM.tif has been created')
else:
  print("Amsterdam_DTM.tif already exists")

# Open the file and calculate the median 'ground' height which is used to calculate building height
with rasterio.open(DTM_merged_path, "r+") as DTM_tile:
  DTM_data = DTM_tile.read(1).astype(rasterio.float32)

  # Change NoData so an accurate mean can be calculated
  DTM_data[DTM_data > 1e+10] = np.nan

  # Calculate DTM mean
  DTM_median = np.nanmedian(DTM_data)

## **Calculate roof greening potential score**

In [ ]:
# Define the roof thresholds (height: m | slope: deg | age: yrs | area: sq m)
height_threshold = 40
slope_threshold = 30
age_threshold = 200

# Add median height to the gdf
bag_median_height_path = os.path.join('roofs', 'BAG_Amsterdam_gdf_median_height.gpkg')

if os.path.isfile(bag_median_height_path):
  print("Reading Amsterdam BAG (including median height column)")
  BAG_Amsterdam_gdf = gpd.read_file(bag_median_height_path)
else:
  print("Calculating median height for every building. This might take around 30 minutes")
  height_stats = zonal_stats(BAG_Amsterdam_gdf, DSM_merged_path, stats="median", nodata=None)
  #BAG_Amsterdam_gdf['median_height'] = [stat['median'] - DTM_median if stat['median'] is not None else None for stat in height_stats]
  BAG_Amsterdam_gdf['median_height'] = None

  for i in range(len(height_stats)):
    if height_stats[i]['median'] != None:
      BAG_Amsterdam_gdf['median_height'].iloc[i] = height_stats[i]['median'] - DTM_median

  # Save it as an intermediate file
  BAG_Amsterdam_gdf.to_file(bag_median_height_path, driver="GPKG")

# Create a slope TIF file to calculate average slope values
input_DSM = gdal.Open(DSM_merged_path)
slope_path = os.path.join("roofs", "Slope.tif")

# Calculate roof slope raster
if not os.path.isfile(slope_path):
  gdal.DEMProcessing(slope_path, input_DSM, "slope", computeEdges=True)
  print("Slope file has been created")
else:
  print("Slope file already exists")

# Add median slope to the gdf
bag_height_slope_path = os.path.join("roofs", "BAG_Amsterdam_gdf_height_slope.gpkg")

if os.path.isfile(bag_height_slope_path):
  print("Reading Amsterdam BAG (including height/slope columns)")
  BAG_Amsterdam_gdf = gpd.read_file(bag_height_slope_path)
else:
  print("Calculating median slope for every building. This might take around 30 minutes")
  slope_stats = zonal_stats(BAG_Amsterdam_gdf, slope_path, stats="median", nodata=None)
  BAG_Amsterdam_gdf['median_slope'] = [stat['median'] if stat is not None else None for stat in slope_stats]

  # Save it as an intermediate file
  BAG_Amsterdam_gdf.to_file(bag_height_slope_path, driver="GPKG")

# Calculate building age (based on build year). Assume the current year is 2024
# We used BAG-light of January 4 2024
current_year = 2024
BAG_Amsterdam_gdf['building_age'] = current_year - BAG_Amsterdam_gdf['bouwjaar']

# Get the roof area from the building geometry
BAG_Amsterdam_gdf['roof_area'] = BAG_Amsterdam_gdf['geometry'].area

# Add the normalized score columns
normalize_df_column(BAG_Amsterdam_gdf, 'building_age',  'norm_age', invert=True)
normalize_df_column(BAG_Amsterdam_gdf, 'roof_area',     'norm_area')
normalize_df_column(BAG_Amsterdam_gdf, 'median_height', 'norm_height', invert=True)
normalize_df_column(BAG_Amsterdam_gdf, 'median_slope',  'norm_slope', invert=True)

# Add weights to the scores (sum should equal 1)
w_age     = 0.25
w_area    = 0.25
w_height  = 0.25
w_slope   = 0.25

# Calculate Greening Potential Score for roofs
BAG_Amsterdam_gdf['GPS_roof'] = w_age    * BAG_Amsterdam_gdf['norm_age']    + \
                                w_area   * BAG_Amsterdam_gdf['norm_area']   + \
                                w_height * BAG_Amsterdam_gdf['norm_height'] + \
                                w_slope  * BAG_Amsterdam_gdf['norm_slope']

# Filter out each building with unsuitable roofs for greening
BAG_Amsterdam_gdf['suitable'] = False
filter_condition = ((BAG_Amsterdam_gdf['building_age']   < age_threshold)      &
                    (BAG_Amsterdam_gdf['median_height']  < height_threshold)   &
                    (BAG_Amsterdam_gdf['median_slope']   < slope_threshold))

BAG_Amsterdam_gdf.loc[filter_condition, 'suitable'] = True
Filtered_BAG_Amsterdam = BAG_Amsterdam_gdf.loc[BAG_Amsterdam_gdf['suitable'] == True].copy()

# Save the FILTERED Amsterdam BAG as a geopackage
Filtered_BAG_Amsterdam_path = os.path.join('roofs', 'Filtered_BAG_Amsterdam.gpkg')

if not os.path.isfile(Filtered_BAG_Amsterdam_path):
  print("Saving filtered Amsterdam BAG")
  Filtered_BAG_Amsterdam.to_file(Filtered_BAG_Amsterdam_path, driver="GPKG")

# Save the UNFILTERED Amsterdam BAG as a geopackage
Unfiltered_BAG_Amsterdam_path = os.path.join('roofs', 'Unfiltered_BAG_Amsterdam.gpkg')

if not os.path.isfile(Unfiltered_BAG_Amsterdam_path):
  print("Saving unfiltered Amsterdam BAG")
  BAG_Amsterdam_gdf.to_file(Unfiltered_BAG_Amsterdam_path, driver="GPKG")

print("Finished!")